In [9]:
import pickle 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [10]:
with open('preprocessed_data/preprocessed_general.pkl', 'rb') as f:
    chosen_stocks = pickle.load(f)

We now have our stock data with features for each stock. The next step for us is to split the data into train and test/validation to fit a scaler only on the training set.
We do that so that the scaler does not get any information about the range the values may lie in in the test or validation set.

In [11]:
end_train_set = chosen_stocks[0][0].index[int((chosen_stocks[0][0].shape[0] - 1) * 0.8)]
end_validation_set = chosen_stocks[0][0].index[int((chosen_stocks[0][0].shape[0] - 1) * 0.9)]
print(end_train_set)
print(end_validation_set)

2024-05-13 00:00:00
2024-11-05 00:00:00


*** WARNING *** This Pipeline somehow confuses the column symbol with usd_jpy. Still to fix.

In [ ]:
# Create lists to hold the data all bins
preprocessed_NN_data = []
for i in range(len(chosen_stocks)):
    preprocessed_NN_bin = []
    for j in range(len(chosen_stocks[i])):
        #Split into train, validation and test sets (data is already sorted by date)
        X_train = chosen_stocks[i][j].loc[:end_train_set, chosen_stocks[i][j].columns != 'target']
        y_train = chosen_stocks[i][j].loc[:end_train_set, 'target']
        X_validation = chosen_stocks[i][j].loc[end_train_set:end_validation_set, chosen_stocks[i][j].columns != 'target']
        y_validation = chosen_stocks[i][j].loc[end_train_set:end_validation_set, 'target']
        X_test = chosen_stocks[i][j].loc[end_validation_set:, chosen_stocks[i][j].columns != 'target']
        y_test = chosen_stocks[i][j].loc[end_validation_set:, 'target']

        # Scale Features based on train set
        # We drop the 'symbol' column before scaling and store it seperately
        symbol = X_train.iloc[0]['symbol']
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train.drop(columns=['symbol'], inplace=False))
        X_validation_scaled = scaler.transform(X_validation.drop(columns=['symbol'], inplace=False))
        X_test_scaled = scaler.transform(X_test.drop(columns=['symbol'], inplace=False))

        # Put back into dfs so we have the original indices and columns for windowing
        columns = X_train.drop(columns=['symbol']).columns
        X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=columns)
        X_validation_scaled = pd.DataFrame(X_validation_scaled, index=X_validation.index, columns=columns) 
        X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=columns)

        # We concatenate the dataframes to create a single dataset for windowing so that we dont lose window.size() windows at the start of each set but only once.
        X = pd.concat([X_train_scaled, X_validation_scaled, X_test_scaled])
        y = pd.concat([y_train, y_validation, y_test])

        entry = (X, y, symbol)
        preprocessed_NN_bin.append(entry)
    preprocessed_NN_data.append(preprocessed_NN_bin)

# Save the data
with open(f'preprocessed_data/preprocessed_LSTM_CNN.pkl', 'wb') as f:
    pickle.dump(preprocessed_NN_data, f)

In [13]:
with open('preprocessed_data/preprocessed_LSTM_CNN.pkl', 'rb') as f:
    LSTM_CNN_preprocessed_data = pickle.load(f)

In [14]:
print(X_train.shape, X_validation.shape, X_test.shape)

(981, 61) (123, 61) (124, 61)
